In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import xarray as xr

In [27]:
pd.read_csv('/glade/work/guoqiang/CTSM_CAMELS/data_paramcailb/ParamCalib_basinparam_6431500.csv')

,Unnamed: 0,Type,Process,PPE_DailyQ_Rank,Parameter,Default_glob,Lower_glob,Upper_glob,Lower_raw,Upper_raw,LowUp_vsPPE,Source,Binding,Method,Default,Lower,Upper
0,0,Hydrology,Infiltration\r\nSoil storage,1,fff,5.000000e-01,2.000000e-02,5.000000e+00,2.000000e-02,5.000000e+00,Same,Param,NaN,Multiplicative,5.000000e-01,2.000000e-02,5.000000e+00
1,1,Hydrology,Infiltration,0,FMAX,3.816694e-01,1.908347e-01,5.725041e-01,1.000000e-01,9.000000e-01,NotInPPE,Surfdata,NaN,Multiplicative,4.055838e-01,2.027919e-01,6.083757e-01
2,2,Hydrology,Infiltration,7,d_max,1.500000e+01,5.000000e+00,1.000000e+02,1.000000e+01,6.000000e+01,BiggerRange_FromYifan,Param,NaN,Multiplicative,1.500000e+01,5.000000e+00,1.000000e+02
3,3,Stomatal resistance and photosynthesis,ET (transpiration),2,medlynslope,4.840380e+00,2.420190e+00,7.260570e+00,3.235949e+00,6.934051e+00,BiggerRange,Param,NaN,Multiplicative,4.840380e+00,2.420190e+00,7.260570e+00
4,4,Hydrology,Soil water drainage,10,hksat_sf,1.000000e+00,1.000000e-01,1.000000e+01,1.000000e-01,1.000000e+01,Same,Param,NaN,Multiplicative,1.000000e+00,1.000000e-01,1.000000e+01
5,5,Hydrology,Soil storage capacity,9,watsat_sf,1.000000e+00,2.500000e-01,2.000000e+00,8.000000e-01,1.200000e+00,BiggerRange,Param,NaN,Multiplicative,1.000000e+00,2.500000e-01,2.000000e+00
6,6,Hydrology,Soil storage capacity,11,sucsat_sf,1.000000e+00,1.000000e-01,1.000000e+01,1.000000e-01,1.000000e+01,Same,Param,NaN,Multiplicative,1.000000e+00,1.000000e-01,1.000000e+01
7,7,Snow Processes,Snow melt,79,n_melt_coef,2.000000e+02,2.500000e+01,6.000000e+02,5.000000e+01,6.000000e+02,BiggerRange_FromYifan,Param,NaN,Multiplicative,2.000000e+02,2.500000e+01,6.000000e+02
8,8,Snow Processes,Snowpack physics,17,zsno,7.750000e-04,1.000000e-05,7.000000e-02,1.000000e-05,7.000000e-02,Same,Param,NaN,Multiplicative,7.750000e-04,1.000000e-05,7.000000e-02
9,9,Hydrology,Infiltration,23,e_ice,6.000000e+00,1.000000e-01,8.000000e+00,1.000000e-01,8.000000e+00,Low0to0.1,Param,NaN,Multiplicative,6.000000e+00,1.000000e-01,8.000000e+00


In [19]:
pd.read_csv('/glade/work/guoqiang/CTSM_CAMELS/data_paramcailb/ParamCalib_11162500.csv')

,Type,Process,PPE_DailyQ_Rank,Parameter,Default,Lower,Upper,Lower_raw,Upper_raw,LowUp_vsPPE,Source,Binding,Method
0,Hydrology,Infiltration\r\nSoil storage,1,fff,0.500000,0.020000,5.000000,0.020000,5.000000,Same,Param,NaN,Multiplicative
1,Hydrology,Soil water drainage,10,hksat_sf,1.000000,0.100000,10.000000,0.100000,10.000000,Same,Param,NaN,Multiplicative
2,Hydrology,Infiltration,0,FMAX,0.381669,0.190835,0.572504,0.100000,0.900000,NotInPPE,Surfdata,NaN,Multiplicative
3,Snow Processes,Rain/Snow forcing classification,0,precip_repartition_nonglc_all_rain_t,2.000000,0.000000,4.000000,0.000000,4.000000,NotInPPE,Namelist,"precip_repartition_glc_all_rain_t,precip_repar...",Additive
4,Snow Processes,Snow melt,79,n_melt_coef,200.000000,25.000000,600.000000,50.000000,600.000000,BiggerRange_FromYifan,Param,NaN,Multiplicative
5,Hydrology,Soil storage capacity,9,watsat_sf,1.000000,0.250000,2.000000,0.800000,1.200000,BiggerRange,Param,NaN,Multiplicative
6,Stomatal resistance and photosynthesis,ET (transpiration),2,medlynslope,4.840380,2.420190,7.260570,3.235949,6.934051,BiggerRange,Param,NaN,Multiplicative
7,Hydrology,Infiltration,7,d_max,15.000000,5.000000,100.000000,10.000000,60.000000,BiggerRange_FromYifan,Param,NaN,Multiplicative
8,Snow Processes,Snowpack physics,17,zsno,0.000775,0.000010,0.070000,0.000010,0.070000,Same,Param,NaN,Multiplicative
9,Hydrology,Soil storage capacity,0,zbedrock,10.639669,5.319835,15.959504,8.512000,12.768000,BiggerRange,Surfdata,NaN,Multiplicative
